In [1]:
from __future__ import division
from dolfin import *

import pandas as pd
import argparse
import math
import os
import shutil
import sympy
import sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ----------------------------------------------------------------------------
# Parameters for DOLFIN and SOLVER 
# ----------------------------------------------------------------------------
set_log_level(LogLevel.INFO)  # log level
# set some dolfin specific parameters
info(parameters,True)
parameters["form_compiler"]["optimize"] = True
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["representation"] = "uflacs"

# -----------------------------------------------------------------------------
# parameters of the solvers
solver_u_parameters = {"nonlinear_solver": "newton",
                       "newton_solver": {"linear_solver": "mumps",
                                          "maximum_iterations": 100,
                                          "absolute_tolerance": 1e-8,
                                          "relative_tolerance": 1e-6,
                                          "report": True,
                                          "error_on_nonconvergence": True}}                              
# parameters of the PETSc/Tao solver used for the alpha-problem
tao_solver_parameters = {"maximum_iterations": 200,
                         "report": False,
                         "line_search": "more-thuente",
                         "linear_solver": "mumps",
                         "method": "tron",
                         "gradient_absolute_tol": 1e-8,
                         "gradient_relative_tol": 1e-8,
                         "error_on_nonconvergence": True}

In [3]:
filename = "mesh/DIC_running"
#Read mesh
mesh = Mesh()
hdf = HDF5File(mesh.mpi_comm(), filename + ".h5", "r")
hdf.read(mesh, "/mesh", False)
ndim = mesh.topology().dim()

boundaries = MeshFunction("size_t", mesh,1)
hdf.read(boundaries, "/boundaries")

subdomains = MeshFunction("size_t", mesh,2)
hdf.read(subdomains, "/subdomains")

rename_subdomains = MeshFunction("size_t", mesh, 2)
rename_subdomains.set_all(0)
rename_subdomains.array()[subdomains.array()==1] = 1

dx = Measure("dx", domain=mesh, subdomain_data=rename_subdomains)

rename_boundaries = MeshFunction("size_t", mesh, 1)
rename_boundaries.set_all(0)
rename_boundaries.array()[boundaries.array()==2] = 2
rename_boundaries.array()[boundaries.array()==3] = 3
rename_boundaries.array()[boundaries.array()==4] = 4

ds = Measure("ds", domain=mesh, subdomain_data=rename_boundaries)

In [4]:
#material
E = Constant(1900.0)
nu = Constant(0.34)
Gc = Constant(0.12)
ell = Constant(0.012)
ut = Constant(1.0)
k_ell = Constant(1e-6)

In [5]:
Id = Identity(ndim)

n = FacetNormal(mesh)

ex  = Constant((1.0, 0.0))

ey  = Constant((0.0, 1.0))

def u_x(u):
    return u.dx(0)

def utt(u):
    return grad(u).T

# -----------------------------------------------------------------------------
# Strain and stress and Constitutive functions of the damage model
# -----------------------------------------------------------------------------
# Strain and stress
def eps(v):
    return sym(grad(v))

def sigma_0(v):
    mu    = E/(2.0*(1.0 + nu))
    lmbda = E*nu/(1.0 - nu**2) # plane stress
    return 2.0*mu*eps(v) + lmbda*tr(eps(v))*Id

# Constitutive functions of the damage model
def w(alpha):
    return alpha

def a(alpha):
    return (1-alpha)**2

def sigma(u, alpha):
    return (a(alpha)+k_ell)*sigma_0(u)


In [6]:
modelname = "DIC_AT1_l0.012_1900_I_integral"
meshname  = modelname+"-mesh.xdmf"

savedir   = "DIC_isotropy/"+modelname+"/"

if MPI.rank(MPI.comm_world) == 0:
    if os.path.isdir(savedir):
        shutil.rmtree(savedir)

In [7]:
# Create function space for 2D elasticity + Damage
V_u     = VectorFunctionSpace(mesh, "Lagrange", 1, dim = 2)
V_alpha = FunctionSpace(mesh, "Lagrange", 1)

# Define the function, test and trial fields
u       = Function(V_u, name="Displacement")
du      = TrialFunction(V_u)
v       = TestFunction(V_u)
alpha   = Function(V_alpha, name="Damage")
dalpha  = TrialFunction(V_alpha)
beta    = TestFunction(V_alpha)

In [8]:
L = 50.
scale=2e-5

# load csv file XY positions
filename = "xy_position_libre"
DIC_0 = pd.read_csv('/Users/xinyuanzhai/DIC_gradam/CT_35_1/' +  filename +   '.csv')

#convert to mm and then dimensionless with L = 50
DIC_0 = DIC_0*1000*scale/L

filename1 = "charge_libre"
U_xy = pd.read_csv('/Users/xinyuanzhai/DIC_gradam/CT_35_1/' +  filename1 +   '.csv')

#convert to mm then dimensionless with L = 50
U_xy = U_xy*1000*scale/L
U_xy.columns = np.arange(len(U_xy.columns))

#Number of point in this contour
xcord = DIC_0.X
ycord = DIC_0.Y

In [9]:

#number of time step
N_image = int(len(U_xy.columns)/2)
#range(len(xcord)-1)

#define the point around the contour
#eps = 1e-6
point = {}
for j in range(len(xcord)):
    point[j] = CompiledSubDomain("near(x[0], x1) && near(x[1], y1)",x1 = DIC_0.X[j] ,y1 = DIC_0.Y[j])
    

In [10]:
#define chargement for all points on the contour
charge = {}
for i in range(len(xcord)):
    charge[i] = Expression(("ux","uy"), ux=0, uy=0, degree=0)

In [11]:
#define displacement BC for all points on the contour
bc = {} 
bc_u = []

for k in range(len(xcord)):
    
    bc[k] = DirichletBC(V_u,charge[k], point[k], method = "pointwise")
    
    bc_u.append(bc[k])

In [12]:
#boundary condition for damage
bc_a1 = DirichletBC(V_alpha, Constant(0.), boundaries, 2)
bc_a2 = DirichletBC(V_alpha, Constant(0.), boundaries, 3)
bc_a3 = DirichletBC(V_alpha, Constant(0.), boundaries, 4)
bc_alpha = [bc_a1, bc_a2, bc_a3]

In [13]:
elastic_energy = 0.5*inner(sigma(u, alpha), eps(u))*dx

# Weak form of elasticity problem
E_u  = derivative(elastic_energy,u,v)
# Writing tangent problems in term of test and trial functions for matrix assembly
E_du = derivative(E_u, u, du)

dissipated_energy  = 3*Gc/8*(w(alpha)/ell + ell*inner(grad(alpha), grad(alpha)))*dx

damage_functional = elastic_energy + dissipated_energy

# First and second directional derivative wrt alpha
E_alpha = derivative(damage_functional,alpha,beta)
E_alpha_alpha = derivative(E_alpha,alpha,dalpha)


In [14]:
# Variational problem for the displacement
problem_u = NonlinearVariationalProblem(E_u, u, bc_u, J=E_du)
# Set up the solvers                                        
solver_u  = NonlinearVariationalSolver(problem_u)
solver_u.parameters.update(solver_u_parameters)

In [15]:
# --------------------------------------------------------------------
# Implement the box constraints for damage field
# --------------------------------------------------------------------
# Variational problem for the damage (non-linear to use variational inequality solvers of petsc)
# Define the minimisation problem by using OptimisationProblem class

class DamageProblem(OptimisationProblem):

    def __init__(self,f,gradf,alpha,J,bcs):
        OptimisationProblem.__init__(self)
        self.total_energy = f
        self.Dalpha_total_energy = gradf
        self.J_alpha = J
        self.alpha = alpha
        self.bc_alpha = bcs

    def f(self, x):
        self.alpha.vector()[:] = x
        return assemble(self.total_energy)

    def F(self, b, x):
        self.alpha.vector()[:] = x
        assemble(self.Dalpha_total_energy, b)
        for bc in self.bc_alpha:
            bc.apply(b)

    def J(self, A, x):
        self.alpha.vector()[:] = x
        assemble(self.J_alpha, A)
        for bc in self.bc_alpha:
            bc.apply(A)

damage_problem = DamageProblem(damage_functional,E_alpha,alpha,E_alpha_alpha,bc_alpha)

# Set up the solvers                                        
solver_alpha  = PETScTAOSolver()
solver_alpha.parameters.update(tao_solver_parameters)



alpha_lb = interpolate(Expression("0.", degree=0), V_alpha)  # lower bound, set to 0
alpha_ub = interpolate(Expression("1.", degree=0), V_alpha)  # upper bound, set to 1

alpha_0 = interpolate(Expression("0.", degree=0), V_alpha)  # initial (known) alpha

In [16]:
file_u = XDMFFile(MPI.comm_world, savedir+"/u.xdmf")
file_alpha = XDMFFile(MPI.comm_world, savedir+"/alpha.xdmf")
for file in [file_u,file_alpha]:
    file.parameters["flush_output"]=True
    file.parameters["rewrite_function_mesh"]=False

In [22]:
load_multipliers = np.linspace(0, N_image-1, N_image)
energies = np.zeros((len(load_multipliers),4))
iterations = np.zeros((len(load_multipliers),2))
force = np.zeros((len(load_multipliers),2))
SIF = np.zeros((len(load_multipliers),4))


# Numerical parameters of the alternate minimization
maxiteration = 2000
AM_tolerance = 1e-4

In [23]:
#----------------------------------------------#
#-----------Create Auxiliary field ------------#
#----------------------------------------------#

#Auxiliry field for T-stress
mu = float(E/(2.0*(1.0 + nu)))
kappav = float((3.0-nu)/(1.0+nu))
Force = 1
u_aux = Expression(["(-F*(kappa+1)/(pi*8*mu))*2.3*std::log(sqrt((x[0])*(x[0])+x[1]*x[1]))- (F/(pi*4*mu))*sin(atan2(x[1], x[0]))*sin(atan2(x[1], x[0]))",
                    "(-F*(kappa-1)/(pi*8*mu))*atan2(x[1], x[0])+ (F/(pi*4*mu))*sin(atan2(x[1], x[0]))*cos(atan2(x[1], x[0]))"],
                    degree=2, mu=mu, kappa=kappav, F=Force,domain=mesh)

sigmax = Expression((('-F*pow(cos(atan2(x[1], x[0])),3)/(pi*((x[0])*(x[0])+x[1]*x[1]))','-F*pow(cos(atan2(x[1], x[0])),2)*sin(atan2(x[1], x[0]))/(pi*((x[0])*(x[0])+x[1]*x[1]))'),
                ('-F*pow(cos(atan2(x[1], x[0])),2)*sin(atan2(x[1], x[0]))/(pi*((x[0])*(x[0])+x[1]*x[1]))','-F*cos(atan2(x[1], x[0]))*pow(sin(atan2(x[1], x[0])),2)/(pi*((x[0])*(x[0])+x[1]*x[1]))')),degree=2,F=Force,domain=mesh)

In [24]:
#Auxiliry field for K1_K2

#Define auxiliry field for Stress intensity factor for K1
k1_aux1 = 1
    #k2_aux1 = 0
u_kaux1 = Expression(["sqrt(sqrt(x[0]*x[0]+x[1]*x[1])/(2*pi))*(k1_aux1*((kappa-cos(atan2(x[1], x[0])))*cos(atan2(x[1], x[0])/2)/(2*mu)))",
                        "sqrt(sqrt(x[0]*x[0]+x[1]*x[1])/(2*pi))*(k1_aux1*((kappa-cos(atan2(x[1], x[0])))*sin(atan2(x[1], x[0])/2)/(2*mu)))"],
                         degree=2, mu=mu, kappa=kappav , k1_aux1=k1_aux1 , domain=mesh)


#Define auxiliry field for Stress intensity factor for K1
sigma_1 = Expression((('(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k1_aux1*(0.75*cos(atan2(x[1], x[0])/2)+0.25*cos(atan2(x[1], x[0])*5/2)))',
                       '(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k1_aux1*(-0.25*sin(atan2(x[1], x[0])/2)+0.25*sin(atan2(x[1], x[0])*5/2)))'),
                      ('(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k1_aux1*(-0.25*sin(atan2(x[1], x[0])/2)+0.25*sin(atan2(x[1], x[0])*5/2)))',
                        '(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k1_aux1*(1.25*cos(atan2(x[1], x[0])/2)-0.25*cos(atan2(x[1], x[0])*5/2)))')),
                        degree=2,mu=mu, kappa=kappav,k1_aux1=k1_aux1 ,  domain=mesh)
    
    
    
#Define auxiliry field for Stress intensity factor for K2
#k1_aux2 = 0
k2_aux2 = 1
u_kaux2 = Expression(["sqrt(sqrt(x[0]*x[0]+x[1]*x[1])/(2*pi))*(k2_aux2*((2+kappa+cos(atan2(x[1], x[0])))*sin(atan2(x[1], x[0])/2)/(2*mu)))",
                      "sqrt(sqrt(x[0]*x[0]+x[1]*x[1])/(2*pi))*(k2_aux2*((2-kappa-cos(atan2(x[1], x[0])))*cos(atan2(x[1], x[0])/2)/(2*mu)))"],
                        degree=2, mu=mu, kappa=kappav, k2_aux2=k2_aux2, domain=mesh)

#Define auxiliry field for Stress intensity factor for K1
sigma_2 = Expression((('(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k2_aux2*(-1.75*sin(atan2(x[1], x[0])/2)-0.25*sin(atan2(x[1], x[0])*5/2)))',
                       '(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k2_aux2*(0.75*cos(atan2(x[1], x[0])/2)+0.25*cos(atan2(x[1], x[0])*5/2)))'),
                      ('(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k2_aux2*(0.75*cos(atan2(x[1], x[0])/2)+0.25*cos(atan2(x[1], x[0])*5/2)))',
                        '(1/sqrt(2*pi*sqrt(x[0]*x[0]+x[1]*x[1])))*(k2_aux2*(-0.25*sin(atan2(x[1], x[0])/2)+0.25*sin(atan2(x[1], x[0])*5/2)))')),
                        degree=2,mu=mu, kappa=kappav, k2_aux2=k2_aux2, domain=mesh)

In [25]:
#charge[1].ux = U_xy[1].values[1]*ut
        
#charge[1].uy = U_xy[1].values[1]*ut

#solver_u.solve()
#plot(u,mode='displacement')

In [ ]:
for (i_t, t) in enumerate(load_multipliers):
    
    for j in range(len(xcord)):
        
        charge[j].ux = U_xy[t].values[j]*ut
        
        charge[j].uy = U_xy[t+N_image].values[j]*ut
        
    if MPI.rank(MPI.comm_world) == 0:
        print("\033[1;32m--- Starting of Time step {0:2d}: t = {1:4f} ---\033[1;m".format(i_t, t)) 
    # Alternate Mininimization 
    # Initialization
    iteration = 1
    err_alpha = 1.0
    # Iterations
    while err_alpha > AM_tolerance and iteration < maxiteration:
        # solve elastic problem
        solver_u.solve()
        # solve damage problem with box constraint 
        solver_alpha.solve(damage_problem, alpha.vector(), alpha_lb.vector(), alpha_ub.vector())
        # test error
        alpha_error = alpha.vector() - alpha_0.vector()
        err_alpha = alpha_error.norm('linf')
        # monitor the results
        if MPI.rank(MPI.comm_world) == 0:
          print ("AM Iteration: {0:3d},  alpha_error: {1:>14.8f}".format(iteration, err_alpha))

        # update iteration
        alpha_0.assign(alpha)
        
        
        #----------------------------------------------#
        #-------------I-integral T-stress--------------#
        #----------------------------------------------#
        I1  = inner((0.5*inner(sigma(u,alpha),eps(u_aux))+ 0.5*inner(sigmax,eps(u))),n[0])*ds(2) \
        + inner((0.5*inner(sigma(u,alpha),eps(u_aux))+ 0.5*inner(sigmax,eps(u))),n[0])*ds(3) \
        + inner((0.5*inner(sigma(u,alpha),eps(u_aux))+ 0.5*inner(sigmax,eps(u))),n[0])*ds(4)

        I2  = dot(dot(sigma(u,alpha),u_x(u_aux)) + dot(sigmax,u_x(u)),n)*ds(2) \
        + dot(dot(sigma(u,alpha),u_x(u_aux)) + dot(sigmax,u_x(u)),n)*ds(3) \
        + dot(dot(sigma(u,alpha),u_x(u_aux)) + dot(sigmax,u_x(u)),n)*ds(4)

        I_int  = assemble(I1-I2)
    
        #print('I-integral: T-stress = ' + str(I_int))
    
    
        #----------------------------------------------#
        #-------------I-integral KI and KII------------#
        #----------------------------------------------#
        #I_integral for seperating KI

        Ik1_1  = inner((0.5*inner(sigma(u,alpha),eps(u_kaux1))+ 0.5*inner(sigma_1,eps(u))),n[0])*ds(2) \
        + inner((0.5*inner(sigma(u,alpha),eps(u_kaux1))+ 0.5*inner(sigma_1,eps(u))),n[0])*ds(3) \
        + inner((0.5*inner(sigma(u,alpha),eps(u_kaux1))+ 0.5*inner(sigma_1,eps(u))),n[0])*ds(4)



        Ik1_2  = dot((dot(sigma(u,alpha),u_x(u_kaux1)) + dot(sigma_1,u_x(u))),n)*ds(2) \
        + dot((dot(sigma(u,alpha),u_x(u_kaux1)) + dot(sigma_1,u_x(u))),n)*ds(3) \
        + dot((dot(sigma(u,alpha),u_x(u_kaux1)) + dot(sigma_1,u_x(u))),n)*ds(4)


        I_k11  = assemble(Ik1_1-Ik1_2)
        I_k1 = I_k11

        k1_I = I_k1*1/(2*k1_aux1)

    
        #I_integral for seperating KII
        Ik2_1  = inner((0.5*inner(sigma(u,alpha),eps(u_kaux2))+ 0.5*inner(sigma_2,eps(u))),n[0])*ds(2) \
        + inner((0.5*inner(sigma(u,alpha),eps(u_kaux2))+ 0.5*inner(sigma_2,eps(u))),n[0])*ds(3) \
        + inner((0.5*inner(sigma(u,alpha),eps(u_kaux2))+ 0.5*inner(sigma_2,eps(u))),n[0])*ds(4)

        Ik2_2  = dot(dot(sigma(u,alpha),u_x(u_kaux2)) + dot(sigma_2,u_x(u)),n)*ds(2) \
        + dot(dot(sigma(u,alpha),u_x(u_kaux2)) + dot(sigma_2,u_x(u)),n)*ds(3) \
        + dot(dot(sigma(u,alpha),u_x(u_kaux2)) + dot(sigma_2,u_x(u)),n)*ds(4)

        I_k2  = assemble(Ik2_1-Ik2_2)

        k2_I = I_k2*1/(2*k2_aux2)
        
        iteration = iteration + 1
    # updating the lower bound to account for the irreversibility
    alpha_lb.vector()[:] = alpha.vector()
    alpha.rename("Damage", "alpha")
    u.rename("Displacement", "u")
    
    # Dump solution to file 
    file_alpha.write(alpha, t)
    file_u.write(u, t)

    # ----------------------------------------
    # Some post-processing
    # ----------------------------------------
    # Save number of iterations for the time step    
    iterations[i_t] = np.array([t, iteration])


    # Calculate the energies
    elastic_energy_value = assemble(elastic_energy)
    surface_energy_value = assemble(dissipated_energy)
    energies[i_t] = np.array([t, elastic_energy_value, surface_energy_value, elastic_energy_value+surface_energy_value])
    force[i_t] = np.array([t, assemble(sigma(u,alpha)[1,1]*ds(2))])
    
    SIF[i_t] = np.array([t, k1_I, k2_I, I_int])
    
    if MPI.rank(MPI.comm_world) == 0:
        print("\nEnd of timestep {0:3d} with load multiplier {1:4f}".format(i_t, t))
        #print("\nElastic and Surface Energies: [{0:6f},{1:6f}]".format(elastic_energy_value, surface_energy_value))
        print("\nElastic and Surface Energies: [{},{}]".format(elastic_energy_value, surface_energy_value))
        print("-----------------------------------------")
        # Save some global quantities as a function of the time
        np.savetxt(savedir + '/energies.txt', energies)
        np.savetxt(savedir + '/iterations.txt', iterations)
        np.savetxt(savedir + '/force.txt', force)
        np.savetxt(savedir + '/SIF.txt', SIF)

--- Starting of Time step  0: t = 0.000000 ---
AM Iteration:   1,  alpha_error:     0.00000000

End of timestep   0 with load multiplier 0.000000

Elastic and Surface Energies: [0.000010,0.000000]

Elastic and Surface Energies: [1.0178007085231071e-05,0.0]
-----------------------------------------
--- Starting of Time step  1: t = 1.000000 ---
AM Iteration:   1,  alpha_error:     0.00000000

End of timestep   1 with load multiplier 1.000000

Elastic and Surface Energies: [0.000062,0.000000]

Elastic and Surface Energies: [6.222666553801626e-05,0.0]
-----------------------------------------
--- Starting of Time step  2: t = 2.000000 ---
AM Iteration:   1,  alpha_error:     0.00000000

End of timestep   2 with load multiplier 2.000000

Elastic and Surface Energies: [0.000171,0.000000]

Elastic and Surface Energies: [0.0001714363624599161,0.0]
-----------------------------------------
--- Starting of Time step  3: t = 3.000000 ---
AM Iteration:   1,  alpha_error:     0.00000000

End of ti

--- Starting of Time step 22: t = 22.000000 ---
AM Iteration:   1,  alpha_error:     0.01497763
AM Iteration:   2,  alpha_error:     0.00424686
AM Iteration:   3,  alpha_error:     0.00122113
AM Iteration:   4,  alpha_error:     0.00035443
AM Iteration:   5,  alpha_error:     0.00010373
AM Iteration:   6,  alpha_error:     0.00003049

End of timestep  22 with load multiplier 22.000000

Elastic and Surface Energies: [0.011621,0.000046]

Elastic and Surface Energies: [0.011621370449105118,4.587058753054835e-05]
-----------------------------------------
--- Starting of Time step 23: t = 23.000000 ---
AM Iteration:   1,  alpha_error:     0.01803220
AM Iteration:   2,  alpha_error:     0.00626204
AM Iteration:   3,  alpha_error:     0.00218995
AM Iteration:   4,  alpha_error:     0.00076994
AM Iteration:   5,  alpha_error:     0.00027128
AM Iteration:   6,  alpha_error:     0.00009566

End of timestep  23 with load multiplier 23.000000

Elastic and Surface Energies: [0.012700,0.000073]

Ela

AM Iteration:  49,  alpha_error:     0.00594869
AM Iteration:  50,  alpha_error:     0.00554091
AM Iteration:  51,  alpha_error:     0.00516594
AM Iteration:  52,  alpha_error:     0.00482103
AM Iteration:  53,  alpha_error:     0.00450033
AM Iteration:  54,  alpha_error:     0.00420249
AM Iteration:  55,  alpha_error:     0.00392697
AM Iteration:  56,  alpha_error:     0.00367165
AM Iteration:  57,  alpha_error:     0.00343630
AM Iteration:  58,  alpha_error:     0.00321881
AM Iteration:  59,  alpha_error:     0.00301478
AM Iteration:  60,  alpha_error:     0.00282818
AM Iteration:  61,  alpha_error:     0.00265436
AM Iteration:  62,  alpha_error:     0.00249173
AM Iteration:  63,  alpha_error:     0.00233854
AM Iteration:  64,  alpha_error:     0.00219349
AM Iteration:  65,  alpha_error:     0.00205660
AM Iteration:  66,  alpha_error:     0.00192719
AM Iteration:  67,  alpha_error:     0.00180649
AM Iteration:  68,  alpha_error:     0.00169441
AM Iteration:  69,  alpha_error:     0.0

AM Iteration:  16,  alpha_error:     0.01383597
AM Iteration:  17,  alpha_error:     0.01303607
AM Iteration:  18,  alpha_error:     0.01225059
AM Iteration:  19,  alpha_error:     0.01158393
AM Iteration:  20,  alpha_error:     0.01094098
AM Iteration:  21,  alpha_error:     0.01026264
AM Iteration:  22,  alpha_error:     0.00956791
AM Iteration:  23,  alpha_error:     0.00888128
AM Iteration:  24,  alpha_error:     0.00822132
AM Iteration:  25,  alpha_error:     0.00760384
AM Iteration:  26,  alpha_error:     0.00703458
AM Iteration:  27,  alpha_error:     0.00651359
AM Iteration:  28,  alpha_error:     0.00603564
AM Iteration:  29,  alpha_error:     0.00559614
AM Iteration:  30,  alpha_error:     0.00519550
AM Iteration:  31,  alpha_error:     0.00483078
AM Iteration:  32,  alpha_error:     0.00449678
AM Iteration:  33,  alpha_error:     0.00419123
AM Iteration:  34,  alpha_error:     0.00391140
AM Iteration:  35,  alpha_error:     0.00365504
AM Iteration:  36,  alpha_error:     0.0

AM Iteration:  90,  alpha_error:     0.00019200
AM Iteration:  91,  alpha_error:     0.00017735
AM Iteration:  92,  alpha_error:     0.00016381
AM Iteration:  93,  alpha_error:     0.00015132
AM Iteration:  94,  alpha_error:     0.00013978
AM Iteration:  95,  alpha_error:     0.00012912
AM Iteration:  96,  alpha_error:     0.00011928
AM Iteration:  97,  alpha_error:     0.00011019
AM Iteration:  98,  alpha_error:     0.00010180
AM Iteration:  99,  alpha_error:     0.00009404

End of timestep  31 with load multiplier 31.000000

Elastic and Surface Energies: [0.013447,0.012511]

Elastic and Surface Energies: [0.013447053277944845,0.012511124074610736]
-----------------------------------------
--- Starting of Time step 32: t = 32.000000 ---
AM Iteration:   1,  alpha_error:     0.05657523
AM Iteration:   2,  alpha_error:     0.04901150
AM Iteration:   3,  alpha_error:     0.04415775
AM Iteration:   4,  alpha_error:     0.04052197
AM Iteration:   5,  alpha_error:     0.03747464
AM Iteration

AM Iteration:  46,  alpha_error:     0.00316986
AM Iteration:  47,  alpha_error:     0.00303374
AM Iteration:  48,  alpha_error:     0.00289533
AM Iteration:  49,  alpha_error:     0.00275569
AM Iteration:  50,  alpha_error:     0.00261843
AM Iteration:  51,  alpha_error:     0.00248469
AM Iteration:  52,  alpha_error:     0.00235338
AM Iteration:  53,  alpha_error:     0.00222547
AM Iteration:  54,  alpha_error:     0.00210068
AM Iteration:  55,  alpha_error:     0.00197960
AM Iteration:  56,  alpha_error:     0.00186416
AM Iteration:  57,  alpha_error:     0.00175431
AM Iteration:  58,  alpha_error:     0.00164876
AM Iteration:  59,  alpha_error:     0.00154750
AM Iteration:  60,  alpha_error:     0.00145068
AM Iteration:  61,  alpha_error:     0.00135842
AM Iteration:  62,  alpha_error:     0.00127072
AM Iteration:  63,  alpha_error:     0.00118744
AM Iteration:  64,  alpha_error:     0.00110866
AM Iteration:  65,  alpha_error:     0.00103431
AM Iteration:  66,  alpha_error:     0.0

AM Iteration:  18,  alpha_error:     0.01300926
AM Iteration:  19,  alpha_error:     0.01230910
AM Iteration:  20,  alpha_error:     0.01163796
AM Iteration:  21,  alpha_error:     0.01099738
AM Iteration:  22,  alpha_error:     0.01037453
AM Iteration:  23,  alpha_error:     0.00975800
AM Iteration:  24,  alpha_error:     0.00915343
AM Iteration:  25,  alpha_error:     0.00857035
AM Iteration:  26,  alpha_error:     0.00801472
AM Iteration:  27,  alpha_error:     0.00749103
AM Iteration:  28,  alpha_error:     0.00700560
AM Iteration:  29,  alpha_error:     0.00656427
AM Iteration:  30,  alpha_error:     0.00616077
AM Iteration:  31,  alpha_error:     0.00581867
AM Iteration:  32,  alpha_error:     0.00550597
AM Iteration:  33,  alpha_error:     0.00517693
AM Iteration:  34,  alpha_error:     0.00483955
AM Iteration:  35,  alpha_error:     0.00450491
AM Iteration:  36,  alpha_error:     0.00417881
AM Iteration:  37,  alpha_error:     0.00386742
AM Iteration:  38,  alpha_error:     0.0